In [1]:
using Plots, Interact
gr()

LoadError: LoadError: ArgumentError: Plots not found in path
while loading In[1], in expression starting on line 1

In [2]:
f(x) = exp(x)
F(x) = exp(x) # Primitiva para fins de comparação
a, b = -1, 1

(-1,1)

In [3]:
plot(f, a, b, c=:red, fill=(0,:red,0.5), leg=false)
h = (b-a)/2
α = (f(a+h)-f(a))/h
β = (f(b)-2*f(a+h)+f(a))/2h^2
plot!(x->f(a) + (x-a)*α + (x-a)*(x-a-h)*β, a, b, c=:blue, fill=(0,:blue,0.5))

LoadError: LoadError: UndefVarError: plot not defined
while loading In[3], in expression starting on line 1

$$
\int_a^b f(x)\mbox{d}x \approx
\frac{h}{3}[f(x_1) + 4f(x_2) + f(x_3)]
$$
com $h = (b-a)/2$, $x_1 = a$, $x_3 = b$.

In [4]:
function simpson(f, a, b)
    return (b-a)*( f(a) + 4*f((a+b)/2) + f(b))/6
end

simpson (generic function with 1 method)

In [5]:
simpson(f, a, b) - (F(b)-F(a))

0.011651369255893052

In [6]:
f(x) = sin(3x) + cos(5x)
@manipulate for m = slider(2:2:100, value=2)
    h = (b-a)/m
    plot(f, a, b, c=:red, fill=(0,:red,0.5), leg=false)    
    for i = 1:2:m-1
        x1, x2, x3 = a + (i-1)*h, a + i*h, a + (i+1)*h
        f1, f2, f3 = f(x1), f(x2), f(x3)
        α = (f2-f1)/h
        β = (f3-2*f2+f1)/2h^2
        plot!(x->f1 + (x-x1)*α + (x-x1)*(x-x2)*β, x1, x3, c=:blue, fill=(0,:blue,0.5))
    end
    plot!()
end

LoadError: LoadError: UndefVarError: @manipulate not defined
while loading In[6], in expression starting on line 13

\begin{align}
\int_a^b f(x)\mbox{d}x & =
\int_{x_1}^{x_m} f(x)\mbox{d}x = 
\int_{x_1}^{x_3} f(x)\mbox{d}x +
\int_{x_3}^{x_5} f(x)\mbox{d}x +
\int_{x_5}^{x_7} f(x)\mbox{d}x + \dots +
\int_{x_{m-2}}^{x_m} f(x)\mbox{d}x \\
& \approx
\frac{h}{3}[f(x_1) + 4f(x_2) + f(x_3)] + 
\frac{h}{3}[f(x_3) + 4f(x_4) + f(x_5)] + \dots +
\frac{h}{3}[f(x_{m-2}) + 4f(x_{m-1}) + f(x_m)] \\
& = \frac{h}{3}
\bigg[ f(x_1) + 4[f(x_2) + f(x_4) + \dots + f(x_{m-1})]
+ 2[f(x_3) + f(x_5) + f(x_7) + \dots + f(x_{m-2})] + f(x_m)\bigg] \\
& = \frac{h}{3}[f(a) + f(b) + 4I_1 + 2I_2]
\end{align}
com $h = (b-a)/m$.

In [7]:
function simpson_repetida(f, a, b; m = 11)
    if m % 2 == 0
        m = m + 1
    end
    h = (b-a)/(m-1)
    I1, I2 = f(a+h), 0.0
    for i = 3:2:m-2
        I1 += f(a + i*h)
        I2 += f(a + (i-1)*h)
    end
    return h*(f(a) + 4I1 + 2I2 + f(b))/3
end

simpson_repetida (generic function with 1 method)

In [8]:
f(x) = exp(x)
simpson_repetida(f, a, b) - (F(b) - F(a))

2.079339388094681e-5

In [9]:
Erro = map(m->simpson_repetida(f, a, b, m=m)-(F(b)-F(a)), 2:100)
plot(2:100, abs(Erro), yaxis=:log)

LoadError: LoadError: UndefVarError: plot not defined
while loading In[9], in expression starting on line 2

In [10]:
function trapezio_repetida(f, a, b; m = 10)
    if m < 2
        error("m deve ser maior ou igual a 2")
    end
    I = f(a) + f(b)
    h = (b-a)/(m-1)
    for i = 1:m-2
        I += 2*f(a + i*h)
    end
    return I * h / 2
end

trapezio_repetida (generic function with 1 method)

In [11]:
Erro = map(m->simpson_repetida(f, a, b, m=m)-(F(b)-F(a)), 2:100)
plot(2:100, abs(Erro), yaxis=:log)
Erro = map(m->trapezio_repetida(f, a, b, m=m)-(F(b)-F(a)), 2:100)
plot!(2:100, abs(Erro), yaxis=:log, c=:red)

LoadError: LoadError: UndefVarError: plot not defined
while loading In[11], in expression starting on line 2

Lembre-se que o erro da aproximação do **trapézio** é de $h^3f''(\xi)$. Daí,
se $f'' = 0$, então o erro é nulo.

In [12]:
f(x) = 1 + x
F(x) = x + x^2/2
trapezio_repetida(f, a, b, m=2) - (F(b)-F(a))

0.0

In [13]:
simpson_repetida(f, a, b, m=2) - (F(b)-F(a))

0.0

In [14]:
f(x) = 1 + x + x^2
F(x) = x + x^2/2 + x^3/3
trapezio_repetida(f, a, b, m=2) - (F(b)-F(a))

1.3333333333333335

In [15]:
simpson_repetida(f, a, b, m=2) - (F(b)-F(a)) # Esperado que seja 0.0

0.0

In [16]:
f(x) = 1 + x + x^2 + x^3
F(x) = x + x^2/2 + x^3/3 + x^4/4
trapezio_repetida(f, a, b, m=2) - (F(b)-F(a))

1.333333333333334

In [17]:
simpson_repetida(f, a, b, m=2) - (F(b)-F(a))

4.440892098500626e-16

In [18]:
f(x) = 1 + x + x^2 + x^3 + x^4
F(x) = x + x^2/2 + x^3/3 + x^4/4 + x^5/5
trapezio_repetida(f, a, b, m=2) - (F(b)-F(a))

2.9333333333333336

In [19]:
simpson_repetida(f, a, b, m=2) - (F(b)-F(a))

0.26666666666666705

Surpreendentemente, o erro da integral de Simpson não é apenas "um a mais" no
expoente, e sim $h^5f^{(iv)}(\xi)$,
de modo que a integral de Simpson **integra polinômios de até grau 3 exatamente**.

In [20]:
1 + sqrt(e/12e-8)

4760.448347286904

In [21]:
f(x) = exp(x)
a, b = 1, 0
F(x) = exp(x)
trapezio_repetida(f, a, b, m=4761) - (F(b)-F(a))

-6.319745171623481e-9

# Quadratura Gaussiana

Como vimos, a ordem do erro de uma integral numérica não é necessariamente a mesma que o número de pontos que são usados.

A quadratura Gaussiana tem por objetivo obter o melhor erro com o menor número de pontos.

Como não podemos calcular o erro exatamente, vamos usar a noção de integrar polinômios exatamente.

Uma quadratura Gaussiana terá a forma

$$ \int_a^b f(x)\mbox{d}x \approx \sum_{i=1}^n A_i f(x_i), $$
onde $n$ é a ordem da quadratura. Para simplificar, vamos começar considerando o caso $[-1,1]$.
$$ \int_{-1}^1\! g(t)\mbox{d}t \approx \sum_{i=1}^n A_i g(t_i), $$

Um polinômio de ordem $N$ geral pode ser escrito como uma combinação dos elementos da base canônica
$1$, $t$, $t^2$, $\dots$, $t^N$. Sendo assim, para interpolar exatamente um polinômio de ordem $N$,
é necessário e suficiente interpolar exatamente todos os $N+1$ elementos de sua base.

Como temos $2n$ incógnitas, o grau máximo do polinômio a ser interpolado é $N = 2n-1$.

Vamos mostrar um exemplo com $n = 1$.

$$ \int_{-1}^1\! g(t)\mbox{d}t \approx A_1 g(t_1), $$

O grau máximo é $N = 1$, então vamos encontrar $A_1$ e $t_1$ tais que
a integral acima é exata para $g(t) = 1$ e $g(t) = t$.

As seguintes equações devem ser satisfeitas:
$$ \int_{-1}^1 1\mbox{d}t = A_1 1 $$
$$ \int_{-1}^1 t\mbox{d}t = A_1 t_1 $$

isto é,
$$ 2 = A_1 $$
$$ 0 = A_1 t_1. $$

A solução é $A_1 = 2$ e $t_1 = 0$.

Isto é,
$$ \int_{-1}^1 g(t)\mbox{d}t \approx 2g(0). $$
Para generalizar para a integral de $f$ em $[a,b]$, basta considerar a mudança
de variável $x = \dfrac{a+b}{2} + \dfrac{b-a}{2}t$.
Daí,
$ \mbox{d}x = \dfrac{b-a}{2}\mbox{d}t$, e então
\begin{align}
\int_a^b f(x) \mbox{d}x
& = \int_{-1}^1 f(x(t)) \frac{b-a}{2} \mbox{d}t \\
& = \frac{b-a}{2}\int_{-1}^1 f\bigg(\frac{a+b}{2} + \frac{b-a}{2}t\bigg)\mbox{d}t \\
& \approx (b-a)f\bigg(\frac{a+b}{2}\bigg).
\end{align}

$$
g(t) = \frac{b-a}{2} f\bigg(\frac{a+b}{2} + \frac{b-a}{2}t\bigg)
= h f(M + ht)
$$

In [23]:
function quadgauss1(f, a, b)
    return (b-a)*f((a+b)/2)
end

quadgauss1 (generic function with 1 method)

In [28]:
a, b = 0, 1
f(x) = exp(x)
F(x) = exp(x)
quadgauss1(f, a, b) - (F(b)-F(a))

-0.0695605577589169

## Exercício

- Fazer com $n = 2$.

In [29]:
function quadgauss2(f, a, b)
    m = (a+b)/2
    h = (b-a)/2
    r = sqrt(3)/3
    return h*(f(m - h*r) + f(m + h*r))
end

quadgauss2 (generic function with 1 method)

In [30]:
f(x) = exp(x)
F(x) = exp(x)
quadgauss2(f, a, b) - (F(b)-F(a))

-0.0003854504515410362

In [31]:
simpson_repetida(f, a, b, m=2) - (F(b)-F(a))

0.0005793234175477391

In [41]:
f(x) = x*log(x)
simpson_repetida(f, 1e-12, 1, m=100)

-0.24999229842028378

In [42]:
quadgauss2(f, 0, 1)

-0.2578533682317772

In [45]:
f(x) = 1/sqrt(1-x^2)
simpson_repetida(f, -1+1e-14, 1-1e-14)

943188.3937531271

In [46]:
quadgauss2(f, 0, 1)

1.3248121441980998

In [63]:
function sinh_tanh(f, a, b; m=10)
    x(t) = tanh(sinh(t))
    w(t) = cosh(t)*sech(sinh(t))^2
    
    g(t) = f(x(t)) * w(t)
    return simpson_repetida(g, -3, 3, m=m)
end

sinh_tanh (generic function with 1 method)

In [74]:
sinh_tanh(f, -1, 1, m=15) - pi

-6.689442549756563e-5